In [100]:
import requests

In [101]:
from bs4 import BeautifulSoup

In [102]:
import json

In [103]:
import datetime

In [104]:
def get_article(url):
    article = dict()
    r = requests.get(url, auth=('user', 'pass'))
    article['soup'] = BeautifulSoup(r.content, 'html.parser')
    article['url'] = url
    article['header'] = article['soup'].title.text
    article['text'] = article['soup'].get_text()
    article['datetime'] = str(datetime.datetime.now())
    return article

In [134]:
def extract_article_links(article, queue, visited):
    article_hrefs = article['soup'].find_all('a', href=True)
    for tag in article_hrefs:
        if tag['href']:
            link = tag['href']
            if link not in visited:
                if 'html' in link:
                    if 'http' in link:
                        queue.add(link)
                    elif 'http' not in link and link[0] == '/':
                        new_link = 'https://www.yahoo.com' + link
                        queue.add(new_link)
                    else:
                        continue
                else:
                    continue
            else:
                continue  
    return queue

In [135]:
from pymongo import MongoClient
network_mdb = '172.18.0.5'
client = MongoClient(network_mdb, 27017)
db = client.test_database
collection = db.test_collection

In [142]:
articles = []
visited = set()
queue = set()
depth = 15
added_articleid = list()

some_url = 'https://www.yahoo.com'
queue.add(some_url)

print('current queue length is' + str(len(queue)))
print('current visited size is' + str(len(visited)))
while depth:
    if queue:
        if not len(visited) % 10:
            print('current queue length is' + str(len(queue)))
            print('current visited size is' + str(len(visited)))
        if len(visited) >= 30:
            print('30 urls visited! Shutting down...')
            break
        article_link = queue.pop()
        visited.add(article_link)
        print('requesting page with URL: ' + article_link)
        article = get_article(article_link)
        print('adding data')
        queue = extract_article_links(article, queue, visited)
    
        if article['url'] != 'https://www.yahoo.com/':
            article.pop('soup')
            added_id = collection.insert_one(article).inserted_id
            added_articleid.append(added_id)
            article.pop('_id')
            articles.append(article)
    depth = depth - 1

current queue length is1
current visited size is0
current queue length is1
current visited size is0
requesting page with URL: https://www.yahoo.com
adding data
requesting page with URL: https://legal.yahoo.com/us/en/yahoo/privacy/dashboard/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/privacy/dashboard/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/privacy/products/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/ip/recidivism/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/permissions/requests/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/privacy/adinfo/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/ip/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en/yahoo/privacy/topics/index.html
adding data
requesting page with URL: https://www.yahoo.com/us/en

In [143]:
len(visited)

15

In [144]:
with open('data.jason', 'w', encoding='utf-8') as w:
  json.dump(articles, w, indent = 4)